In [4]:
def get_data(endpoint, per_page = 100, cursor = None):
    url = "https://api.openalex.org/" + endpoint + "?per-page=" + per_page + "&cursor=" + cursor 
    data = requests.get(url)
    data_dict = json.loads(data.text)
    data_df = pd.DataFrame.from_dict(data_dict["results"])
    return data_df

In [27]:
import math
url = "https://api.openalex.org/sources?per-page=100&cursor=*"

sources = requests.get(url)
sources_dict = json.loads(sources.text)
total_pages = math.ceil(sources_dict["meta"]["count"]/100)
sources_df = pd.DataFrame.from_dict(sources_dict["results"])
sources_df.to_csv("sources.csv", mode='a', index=False)

while(total_pages > 1):
    cursor = sources_dict["meta"]["next_cursor"]
    sources_df = get_data("sources", "100", cursor)
    sources_df.to_csv("sources.csv", mode='a', index=False)
    total_pages -= 1

In [26]:
import math
url = "https://api.openalex.org/publishers?per-page=100&cursor=*"

publisher = requests.get(url)
publisher_dict = json.loads(publisher.text)
total_pages = math.ceil(publisher_dict["meta"]["count"]/100)
publisher_df = pd.DataFrame.from_dict(publisher_dict["results"])
publisher_df.to_csv("publisher.csv", mode='a', index=False)

# while(total_pages > 1):
#     cursor = publisher_dict["meta"]["next_cursor"]
#     publisher_df = get_data("publishers", "100", cursor)
#     publisher_df.to_csv("publisher.csv", mode='a', header=False, index=False)
#     total_pages -= 1

In [39]:
final_list_len = len(final_source)
result_list_len = len(ssci_in_sources)
# find percentage of journals in ssci_journals_in_sources
print("Percentage of journals in SSCI 2022 in OpenAlex: " + percentage(result_list_len, final_list_len))


Percentage of journals in SSCI 2022 in OpenAlex: 13.584905660377359%


In [42]:
print(percentage(ahci_in_sources_len + sce_in_sources_len + result_list_len, final_list_len))

80.75471698113208%


In [40]:
ahci_in_sources_len = len(ahci_in_sources)
print("Percentage of journals in AHCI 2022 in OpenAlex: " + percentage(ahci_in_sources_len, final_list_len))

Percentage of journals in AHCI 2022 in OpenAlex: 9.81132075471698%


In [41]:
sce_in_sources_len = len(sce_in_sources)
print("Percentage of journals in SCE 2022 in OpenAlex: " + percentage(sce_in_sources_len, final_list_len))

Percentage of journals in SCE 2022 in OpenAlex: 57.35849056603774%


In [1]:
import pandas as pd
import requests
import json 
import numpy as np

In [3]:
ssci_dfs = pd.read_excel("./SSCI2022.xlsx", sheet_name=None)
ahci_dfs = pd.read_excel("./AHCI2022.xlsx", sheet_name=None)
sce_dfs = pd.read_excel("./SCE2022.xlsx", sheet_name=None)
sources_dnf = pd.read_csv("./sources.csv")

In [4]:
ahci_journals_issn = ahci_dfs['wos-core_AHCI 2022-June-22']["Column2"].tolist()
ahci_journals_issn.pop(0)
ahci_journals_issn = [x for x in ahci_journals_issn if type(x) == str]
ahci_journals_issne = ahci_dfs['wos-core_AHCI 2022-June-22']["Column3"].tolist()
ahci_journals_issne.pop(0)
ahci_journals_issne = [x for x in ahci_journals_issne if type(x) == str]
ahci_journals_issn.extend(ahci_journals_issne)



In [5]:
sce_journals_issn = sce_dfs['June 22, 2022']["Column2"].tolist()
sce_journals_issn.pop(0)
sce_journals_issn = [x for x in sce_journals_issn if type(x) == str]
sce_journals_issne = sce_dfs['June 22, 2022']["Column3"].tolist()
sce_journals_issne.pop(0)
sce_journals_issne = [x for x in sce_journals_issne if type(x) == str]
sce_journals_issn.extend(sce_journals_issne)


In [6]:
ssci_journals_issn = ssci_dfs['wos-core_SSCI 2022-June-22']["Column2"].tolist()
ssci_journals_issn.pop(0)
ssci_journals_issn = [x for x in ssci_journals_issn if type(x) == str]
ssci_journals_issne = ssci_dfs['wos-core_SSCI 2022-June-22']["Column3"].tolist()
ssci_journals_issne.pop(0)
ssci_journals_issne = [x for x in ssci_journals_issne if type(x) == str]
ssci_journals_issn.extend(ssci_journals_issne)

In [7]:
sources_issn = [x for x in sources_dnf.issn if type(x) == str]

In [136]:
def filter_journals(journal_list):
    filtered_journals = []
    for journal in journal_list:
        for source in sources_issn:
            if journal in source and journal not in filtered_journals:
                filtered_journals.append(journal)
    return filtered_journals

def percentage(part, whole):
    res = 100 * float(part)/float(whole)
    return str(res) + "%"

def diff(raw, filtered):
    return len(raw) - len(filtered)

In [137]:
ssci_in_sources = filter_journals(ssci_journals_issn)
ahci_in_sources = filter_journals(ahci_journals_issn)
sce_in_sources = filter_journals(sce_journals_issn)

In [138]:
dif_ssci = diff(ssci_journals_issn, ssci_in_sources)
dif_ahci = diff(ahci_journals_issn, ahci_in_sources)
dif_sce = diff(sce_journals_issn, sce_in_sources)
print("SSCI total" + str(len(ssci_journals_issn)))
print("SSCI difference: " + str(dif_ssci))
print("AHCI total" + str(len(ahci_journals_issn)))
print("AHCI difference: " + str(dif_ahci))
print("SCE total" + str(len(sce_journals_issn)))
print("SCE difference: " + str(dif_sce))

SSCI total7065
SSCI difference: 7056
AHCI total3490
AHCI difference: 3486
SCE total18901
SCE difference: 18759


In [142]:
print(ssci_journals_issn)
print("044-5851" in sources_issn)
print(ssci_in_sources)


['0001-3072', '1012-8255', '1042-9670', '1941-6520', '2168-1007', '0001-4273', '1537-260X', '1558-9080', '0363-7425', '0001-4575', '0001-4788', '0810-5391', '0951-3574', '0155-9982', '0888-7993', '0361-3682', '0001-4826', '1585-1923', '1726-569X', '1318-0185', '2559-8201', '0001-6373', '0103-2100', '0001-6810', '0001-690X', '0001-6918', '0001-6993', '0335-5322', '1476-7503', '1469-7874', '1059-7123', '0965-2140', '1606-6359', '1940-0640', '0306-4603', '0214-4840', '0894-587X', '0095-3997', '0001-8368', '0001-8392', '2363-8346', '0741-7136', '1895-1171', '1382-4996', '1040-2608', '2515-2459', '1536-0903', '0161-9268', '1527-7941', '2332-8584', '0886-1099', '0001-9720', '0001-9909', '2033-8732', '1569-2094', '0263-0338', '1017-6772', '2374-3670', '0795-4778', '1118-4841', '0002-0184', '0002-0206', '0002-0397', '0144-686X', '1359-1789', '0096-140X', '1360-7863', '1382-5585', '0742-4477', '2193-7532', '0169-5150', '0139-570X', '0002-1482', '0889-048X', '1695-9752', '1090-7165', '0954-0121'

In [9]:
%store ssci_journals_issn
%store ahci_journals_issn
%store sce_journals_issn

Stored 'ssci_journals_issn' (list)
Stored 'ahci_journals_issn' (list)
Stored 'sce_journals_issn' (list)
